In [124]:
import requests
import time
from requests import get
from bs4 import BeautifulSoup

In [125]:
page1 = requests.get('https://www.whitehouse.gov/presidential-actions/president-donald-j-trump-announces-intent-nominate-personnel-key-administration-posts-55/')

In [126]:
soup2 = BeautifulSoup(page1.content, 'html.parser')

In [127]:
html = soup2.find("div", {"class": "page-content"})
ptags = list(html.find_all("p"))

### define func to get urls

In [133]:
def get_Urls(url):
    raw_html = requests.get(url)
    soup = BeautifulSoup(raw_html.content, 'html.parser')
    h2s = soup.find_all("h2")
    link = []
    for h2 in h2s:
        link.append(h2.find("a", href=True))
    if link == []: 
        print(soup)
    return link

### def function to parse a page

In [152]:
def parsePage(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html = soup.find("div", {"class": "page-content"})
    ptags = list(html.find_all("p"))
    str_ptags = []
    for tag in ptags:
        str_ptags.append(str(ptags))
    return str_ptags

### get the number of pages

In [130]:
first_list = requests.get('https://www.whitehouse.gov/presidential-actions/').text
soup3 = BeautifulSoup(first_list, 'html.parser')
pageNum = soup3.find("div", {"class": "pagination"})
pageList = []
for atags in pageNum.find_all("a", class_=['page-numbers']):
    pageList.append(str(atags.get_text()))
    
firstList = pageList[0]
varList = pageList[-1]


### scrape through all the lists of urls pages

In [138]:
nominations_urls = []

# can put in int(varList) +1 for end of range if want all pages, but that's returning 404 err starting on pg 7

for index in range(1, 6):
    url = 'https://www.whitehouse.gov/presidential-actions/page/' + str(index)
    print(url)
    list_of_urls = get_Urls(url + str(index))
    nominations_urls = nominations_urls + list_of_urls
    time.sleep(3)
print(nominations_urls)

https://www.whitehouse.gov/presidential-actions/page/1
https://www.whitehouse.gov/presidential-actions/page/2
https://www.whitehouse.gov/presidential-actions/page/3
https://www.whitehouse.gov/presidential-actions/page/4
https://www.whitehouse.gov/presidential-actions/page/5
[<a href="https://www.whitehouse.gov/presidential-actions/president-donald-j-trump-proclaims-may-15-2018-peace-officers-memorial-day-may-13-may-19-2018-police-week/">President Donald J. Trump Proclaims May 15, 2018, as Peace Officers Memorial Day and May 13 through May 19, 2018, as Police Week</a>, <a href="https://www.whitehouse.gov/presidential-actions/president-donald-j-trump-proclaims-may-13-may-19-2018-national-transportation-week/">President Donald J. Trump Proclaims May 13 through May 19, 2018, as National Transportation Week</a>, <a href="https://www.whitehouse.gov/presidential-actions/president-donald-j-trump-announces-intent-appoint-personnel-key-administration-post-3/">President Donald J. Trump Announces 

### go through noms urls and get the p tags

In [168]:
all_noms_texts = []

for url in nominations_urls:
    print(url.text)
    ptags = parsePage(url["href"])
    d = {}
    d["hed"] = url.text
    d["html"] = "".join(ptags)[1:-1]
    all_noms_texts.append(d)
    time.sleep(3)
    
#     trying to get list into string so it can be read as html

President Donald J. Trump Proclaims May 15, 2018, as Peace Officers Memorial Day and May 13 through May 19, 2018, as Police Week
President Donald J. Trump Proclaims May 13 through May 19, 2018, as National Transportation Week
President Donald J. Trump Announces Intent to Appoint Personnel to Key Administration Post
President Donald J. Trump Proclaims May 13, 2018, as Mother’s Day
President Donald J. Trump Announces Intent to Nominate and Appoint Personnel to Key Administration Posts
President Donald J. Trump Proclaims May 11, 2018, as Military Spouse Day
Five Nominations and Two Withdrawals Sent to the Senate Today
President Donald J. Trump Announces Fourteenth Wave of Judicial Nominees, Thirteenth Wave of United States Attorney Nominees, and Eighth Wave of United States Marshal Nominees
Executive Order Enhancing Noncompetitive Civil Service Appointments of Military Spouses
Ceasing U.S. Participation in the JCPOA and Taking Additional Action to Counter Iran’s Malign Influence and Deny 

In [170]:
all_noms_texts[-1]

{'hed': 'Three Nominations Sent to the Senate Today',
 'html': '<p>NOMINATIONS SENT TO THE SENATE:</p>, <p>Claire M. Grady, of Pennsylvania, to be Under Secretary for Management, Department of Homeland Security, vice Russell C. Deyo.</p>, <p>Neomi Rao, of the District of Columbia, to be Administrator of the Office of Information and Regulatory Affairs, Office of\xa0Management and Budget, vice Howard A. Shelanski.</p>, <p>David J. Redl, of New York, to be Assistant Secretary of\xa0Commerce for Communications and Information, vice Lawrence E.\xa0Strickling.</p>][<p>NOMINATIONS SENT TO THE SENATE:</p>, <p>Claire M. Grady, of Pennsylvania, to be Under Secretary for Management, Department of Homeland Security, vice Russell C. Deyo.</p>, <p>Neomi Rao, of the District of Columbia, to be Administrator of the Office of Information and Regulatory Affairs, Office of\xa0Management and Budget, vice Howard A. Shelanski.</p>, <p>David J. Redl, of New York, to be Assistant Secretary of\xa0Commerce for

In [173]:
with open("NominationAnnouncements.json", "w") as ofile:
    ofile.write(str(all_noms_texts))
ofile.close()